## Welcome!
to the repo for

*Learning the Legibility of Visual Text Perturbations* (EACL 2023)

by Dev Seth, Rickard Stureborg, Danish Pruthi and Bhuwan Dhingra

### A `LEGIT` Introduction
This notebook provides a helpful starting point to interact with the datasets and models presented in the Learning Legibility paper.

All assets are hosted on the HuggingFace Hub and can be used with the `transformers` and `datasets` libraries: 
  - TrOCR-MT Model: https://huggingface.co/dvsth/LEGIT-TrOCR-MT 
  - LEGIT Dataset: https://huggingface.co/datasets/dvsth/LEGIT
  - Perturbed Jigsaw Dataset: https://huggingface.co/datasets/dvsth/LEGIT-VIPER-Jigsaw-Toxic-Comment-Perturbed

##### Setup

In [8]:
# external imports -- use pip or conda to install these packages
import torch
from transformers import TrOCRProcessor, AutoModel, TrainingArguments
from datasets import load_dataset

# local imports
from classes.LegibilityModel import LegibilityModel
from classes.Trainer import MultiTaskTrainer
from classes.Metrics import binary_classification_metric, ranking_metric

#### Loading the Model and Dataset

In [9]:
# load the model schema and pretrained weights
# (this may take some time to download)
model = AutoModel.from_pretrained("dvsth/LEGIT-TrOCR-MT", revision='main', trust_remote_code=True)

loading configuration file config.json from cache at /Users/dvsth/.cache/huggingface/hub/models--dvsth--LEGIT-TrOCR-MT/snapshots/eb8f84a70f3e400077663e48d497ab7ce796c724/config.json
Model config VisionEncoderDecoderConfig {
  "_commit_hash": "eb8f84a70f3e400077663e48d497ab7ce796c724",
  "_name_or_path": "dvsth/LEGIT-TrOCR-MT",
  "architectures": [
    "LegibilityModel"
  ],
  "auto_map": {
    "AutoModel": "LegibilityModel.LegibilityModel"
  },
  "decoder": {
    "_name_or_path": "",
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": 768,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 12,
    "decoder_start_token

Interactive dataset preview available [here](https://huggingface.co/datasets/dvsth/LEGIT/viewer/dvsth--LEGIT/test).

In [10]:
dataset = load_dataset('dvsth/LEGIT').with_format('torch')

Using custom data configuration dvsth--LEGIT-d84a4d72774d3652
Found cached dataset parquet (/Users/dvsth/.cache/huggingface/datasets/dvsth___parquet/dvsth--LEGIT-d84a4d72774d3652/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

#### Training/Eval Loop

##### Trainer setup

In [11]:
# preprocessor provides image normalization and resizing
preprocessor = TrOCRProcessor.from_pretrained(
    "microsoft/trocr-base-handwritten")

# apply preprocessing batch-wise
def collate_fn(data):
    return {
        'choice': torch.tensor([d['choice'].item() for d in data]),
        'img0': preprocessor([d['img0'] for d in data], return_tensors='pt')['pixel_values'],
        'img1': preprocessor([d['img1'] for d in data], return_tensors='pt')['pixel_values']
    }


train_args = TrainingArguments(
    output_dir=f'runs',             # change this to a unique path for each run, e.g. f'runs/{run_id}'
    overwrite_output_dir=True,
    num_train_epochs=5,             # we found 3 epochs to be sufficient for convergence on the base models
    per_device_train_batch_size=26, # fits on 1 x NVIDIA A6000, 48GB VRAM
    per_device_eval_batch_size=26,  # can be increased to 32
    gradient_accumulation_steps=2,  # increase this to fit on a smaller GPU
    warmup_steps=0,             
    weight_decay=0.0,
    learning_rate=1e-5,             # we found this to be the best initial learning rate for the base models
    save_strategy="steps",
    save_steps=200,
    eval_steps=200,
    evaluation_strategy="steps",
    logging_strategy='steps',
    logging_steps=50,
    fp16=False,                     
    load_best_model_at_end=True,    # load the best model at the end of training based on validation F1
    metric_for_best_model='f1_score')

trainer = MultiTaskTrainer(
    model=model,
    compute_metrics=binary_classification_metric, # check out metrics.py for a list of metrics
    args=train_args,
    data_collator=collate_fn,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'])


loading configuration file preprocessor_config.json from cache at /Users/dvsth/.cache/huggingface/hub/models--microsoft--trocr-base-handwritten/snapshots/0cc7abaad739c7902dabd562356c7e6a7be834ea/preprocessor_config.json
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
{param_name} should be a dictionary on of the following set of keys: {VALID_SIZE_DICT_KEYS}, got {size}. Converted to {size_dict}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}

loading file vocab.json from cache at /Users/dvsth/.cache/huggingface/hub/models--microsoft--trocr-base-handwritten/s

##### Generate predictions and compute metrics

In [12]:
predictions = trainer.predict(dataset['test'].select(range(100))) # takes ~1-2 minutes on a laptop CPU
print(predictions.metrics)

The following columns in the test set don't have a corresponding argument in `LegibilityModel.forward` and have been ignored: n1, word, k, k1, n, word1, model1, model0, word0. If n1, word, k, k1, n, word1, model1, model0, word0 are not expected by `LegibilityModel.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 26


  0%|          | 0/4 [00:00<?, ?it/s]

{'test_loss': 0.5344929695129395, 'test_precision': 0.9479166567925349, 'test_recall': 0.8921568539984622, 'test_accuracy': 0.8787878721303949, 'test_f1_score': 0.9191914103665608, 'test_runtime': 47.7405, 'test_samples_per_second': 2.095, 'test_steps_per_second': 0.084}
